In [16]:
import numpy as np
import pandas as pd

from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier


from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing  import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

In [17]:
mean_league_team = pd.read_csv("mean_league_teamv1.csv")
mean_league_team.head(10)

,year,team_placement,age,height,weight
0,2003,1,40.515152,1.831212,74.848485
1,2003,2,43.129032,1.815806,76.709677
2,2003,3,40.000000,1.813590,78.256410
3,2003,4,40.605263,1.828684,75.540541
4,2003,5,41.000000,1.799615,74.000000
5,2003,6,40.607143,1.825926,75.037037
6,2003,7,42.968750,1.805937,75.656250
7,2003,8,42.666667,1.821053,77.631579
8,2003,9,41.909091,1.816061,76.666667
9,2003,10,42.769231,1.820769,76.076923


In [18]:
mean_league_team.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            380 non-null    int64  
 1   team_placement  380 non-null    int64  
 2   age             380 non-null    float64
 3   height          380 non-null    float64
 4   weight          380 non-null    float64
dtypes: float64(3), int64(2)
memory usage: 15.0 KB


## Baseline

In [19]:
# Teile in Train und Test Daten
X = mean_league_team.drop(columns=['year','team_placement'])
y =mean_league_team.team_placement

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.33, random_state=42)

In [20]:
# Baseline 
LR = LogisticRegression(random_state=0)
LR.fit(X_treino, y_treino)

LogisticRegression(random_state=0)

In [21]:
## score vai de 0 a 1 (0% - 100%)
LR.score(X_teste, y_teste)*100

1.5873015873015872

Unsere Baseline zeigt eine Acccuracy von 1,58 %. Das war ein unerwartetes Ergebnis. Wir müssen in dieser Stelle versuchen, eine andere Vorgehensweise, um diese Ergebnisse zu verbessern. Wir werden die Spalten Age, height und weigth in derselben Skala bringen und ein Kreuz Validierungsverfahren durchführen.

**Min-Max-Skalierung:**

- Bei unser  Eingabevariablen haben wir unterschiedliche Einheiten haben (z. B. Jahren, Meter und Kilo), was wiederum bedeuten kann, dass die Variablen unterschiedliche Skalen haben. Unterschiede in den Skalen zwischen den Eingabevariablen können den Schwierigkeitsgrad des zu modellierenden Problems erhöhen. Ein Beispiel hierfür ist, dass große Eingabewerte (z. B. eine Spanne von Hunderten oder Tausenden von Einheiten) zu einem Modell führen können, das große Gewichtungswerte lernt. Ein Modell mit großen Gewichtungwerten ist häufig instabil, was bedeutet, dass es während des Lernens unter schlechter Leistung und Empfindlichkeit gegenüber Eingabewerten leiden kann, was zu höheren Generalisierungsfehlern führt.

<br>

- **Gradientenbasierte Optimierungsalgorithmen**
Modelle wie **lineare Regression, logistische Regression, neuronale Netze und Support-Vektor-Maschinen (SVMs)** verwenden gradientenbasierte Optimierungsalgorithmen (z. B. Gradientenabstieg), um ihre Parameter zu schätzen. Wenn die Features unterschiedliche Skalen haben, kann dies die Konvergenz des Optimierungsalgorithmus verlangsamen und zu einer suboptimalen Lösung führen. Beispiel: Wenn ein Feature in Metern und ein anderes in Millimetern gemessen wird, kann das Feature in Millimetern die Gradienten dominieren und das Modell in die falsche Richtung führen.

<br>

- **Distance-based Models
Modelle wie k-nearest neighbors (k-NN), k-means clustering und Principal Component Analysis (PCA)** basieren auf Abständen oder Distanzen zwischen Datenpunkten. Unterschiede in den Skalen der Features können dazu führen, dass einige Features übermäßig starken Einfluss auf die Berechnung der Distanzen haben. Beispiel: Wenn ein Feature im Bereich von 1 bis 1000 variiert und ein anderes im Bereich von 1 bis 10, wird das erste Feature die Distanzmetrik dominieren.

**Min-Max-Skalierung:**
- Skaliert die Daten auf einen Bereich von 0 bis 1 (oder einen anderen spezifizierten Bereich).

<br>


\begin{align}
Xskaled = (\frac{X-\ Xmin}{Xmax - Xmin})
\end{align}

<br>
<br>

**Validierungsverfahren**

- Die Funktion cross_validate_model führt eine Kreuzvalidierung durch, um die Leistung verschiedener Modelle auf einem gegebenen Datensatz zu bewerten. Dabei wird die KFold-Methode verwendet, um die Daten in mehrere Folds aufzuteilen. Für jedes Modell wird die durchschnittliche Genauigkeit sowie die Standardabweichung dieser Genauigkeit berechnet.

- KFold generiert Indizes der Zeilen für jede Aufteilung, die dann für das Training und die Validierung genutzt werden. Die Anzahl der Folds (n_splits) ist auf 5 festgelegt, was bedeutet, dass die Daten in fünf Teile aufgeteilt werden. Dabei sorgt shuffle=True dafür, dass die Daten vor der Aufteilung zufällig durchmischt werden.

- Der Min-Max-Scaler wird verwendet, um die Merkmale des Datensatzes auf denselben Wertebereich zu skalieren, was sicherstellt, dass sie vergleichbar sind und das Modell nicht durch unterschiedliche Skalen beeinträchtigt wird.

- Für jedes Modell werden die Ergebnisse über die verschiedenen Folds hinweg berechnet, wobei insgesamt 5 Wiederholungen der Kreuzvalidierung durchgeführt werden. Der Mittelwert der Genauigkeiten wird schließlich als Ergebnis betrachtet.

RepeatedKFold ist eine Methode zur Kreuzvalidierung, die verwendet wird, wenn Sie die Stabilität und Robustheit Ihrer Modelle genauer überprüfen möchten. Sie ist besonders nützlich in den folgenden Situationen:

**1. Kleine Datensätze**
Bei kleinen Datensätzen können zufällige Aufteilungen in Trainings- und Validierungssätze stark unterschiedliche Ergebnisse liefern. RepeatedKFold hilft dabei, diese Variabilität zu reduzieren, indem sie mehrere K-Fold-Kreuzvalidierungen durchführt und die Ergebnisse mittelt.

**2. Modellvergleich**
Wenn Sie verschiedene Modelle vergleichen möchten, bietet RepeatedKFold eine robustere Bewertung, da die Modelle über mehrere Wiederholungen hinweg getestet werden. Dies reduziert die Wahrscheinlichkeit, dass die Leistung eines Modells zufällig besser oder schlechter erscheint als in der Realität.

**3. Bewertung der Modellstabilität**
Wenn Sie wissen möchten, wie stabil die Leistung Ihres Modells ist, kann RepeatedKFold durch mehrere Wiederholungen zeigen, wie sehr die Ergebnisse variieren. Ein Modell, das in allen Wiederholungen ähnlich gute Ergebnisse liefert, ist stabiler als ein Modell, dessen Leistung stark schwankt.

**4. Schätzungen mit geringerer Varianz**
Durch die Wiederholung des K-Fold-Verfahrens wird die Varianz der Leistungsschätzungen reduziert, was zu genaueren und zuverlässigeren Schätzungen führt.


RepeatedKFold ist ein leistungsfähiges Werkzeug zur Verbesserung der Zuverlässigkeit und Stabilität von Modellbewertungen, insbesondere bei kleinen Datensätzen oder wenn eine genaue Modellvergleichung erforderlich ist. Es kann helfen, die Varianz der Ergebnisse zu reduzieren und eine genauere Einschätzung der Modellleistung zu liefern.

### Vorbereitung der Daten für RepeatedKFold

In [22]:
# Vorbereitung von den Datest für RepeatedKFold: Für  Daten importiren und zeigen
mean_league_team = pd.read_csv("mean_league_teamv1.csv")
mean_league_team.head(10)

,year,team_placement,age,height,weight
0,2003,1,40.515152,1.831212,74.848485
1,2003,2,43.129032,1.815806,76.709677
2,2003,3,40.000000,1.813590,78.256410
3,2003,4,40.605263,1.828684,75.540541
4,2003,5,41.000000,1.799615,74.000000
5,2003,6,40.607143,1.825926,75.037037
6,2003,7,42.968750,1.805937,75.656250
7,2003,8,42.666667,1.821053,77.631579
8,2003,9,41.909091,1.816061,76.666667
9,2003,10,42.769231,1.820769,76.076923


In [23]:
# Die Daten werden aufgeteilt in 20 % Testen und 80 % Training. 
# Die Testdaten werden nur zum Schluss verwendet und werden die Realität simulieren
train, test = train_test_split(mean_league_team, test_size=0.40, random_state=42)


# Train und Valiedirung  year wird raus genommen wegen die Multikollinearität mit und age 
x = train.drop(columns=['year','team_placement'])
y = train.drop(columns=['year','age','height','weight'])


# reset index 
x= x.reset_index(drop=True)
y= y.reset_index(drop=True)


In [24]:
# X Variable Überprüfung
x

,age,height,weight
0,39.878788,1.812812,74.468750
1,36.756757,1.817568,77.108108
2,37.131579,1.821579,76.342105
3,32.048780,1.798000,74.075000
4,36.892857,1.834286,75.178571
...,...,...,...
223,37.027027,1.830278,76.500000
224,39.000000,1.834688,77.937500
225,29.636364,1.831290,76.677419
226,25.000000,1.799259,72.238095


In [25]:
# Y Variable Überprüfung
y

,team_placement
0,10
1,17
2,8
3,12
4,13
...,...
223,12
224,7
225,11
226,9


In [26]:
# Kreuzvalidierungsfunktion für Klassifikation
def cross_validate_model(models, x, y, n_splits=5):
    
    # Dict für die Ergbnisse
    results = {}
    scaler = MinMaxScaler()
    
    for name, model in models:
        accuracy_scores = []
        
         # KFold teilt die Daten in n_splits Folds auf.
         # Wiederholt die K-fache Aktion n-mal mit unterschiedlicher Zufallsverteilung bei jeder Wiederholung.
         # shuffle=True stellt sicher, dass die Daten vor der Aufteilung gemischt werden.
         # random_state=rep Die beispiele für jeden Block werden sich ändern 
         # Arlternative lösung  kf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=2652124)
        for rep in range(10):
            kf = KFold(n_splits=n_splits, shuffle=True, random_state=rep)
        
            # Durchführen der Kreuzvalidierung
            for train_index, val_index in kf.split(x):
                # Aufteilen der Daten in Trainings- und Validierungs-Sets, Hier werden in den Index zugreifen.:
                x_train, x_val = x.iloc[train_index], x.iloc[val_index]
                y_train, y_val = y.iloc[train_index], y.iloc[val_index]

                # Anwenden des Scalers , da die Skala von den Datensets sehr unterschiedlich ist.
                x_train = scaler.fit_transform(x_train)
                x_val = scaler.transform(x_val)

                # LabelEncoder für "XGBClassifier"
                # XGBClassifier muss eine  Klassenspalte bei 0 beginnen (wie seit Version 1.3.2 erforderlich).
                # Eine einfache Lösung hierfür ist die Verwendung von LabelEncoder aus der Bibliothek sklearn.preprocssing.
                # Dies liegt daran, dass der y_train vor dem Training in einem 
                # neueren aktualisierten XGBoost-Modell codiert werden muss. 
                # Das heißt, Sie müssen eine kategorische Transformation wie Label-Encoder 
                le = LabelEncoder()
                y_train = le.fit_transform(y_train)
                
                # Anwenden des Scalers
                x_train = scaler.fit_transform(x_train)
                x_val = scaler.transform(x_val)

                model.fit(x_train, y_train)
                y_pred = model.predict(x_val)

                # Berechnung der Genauigkeit
                accuracy = accuracy_score(y_val, y_pred)
                accuracy_scores.append(accuracy)
           
        # Accuracy und Standardabweichung in eine Liste speichern
        mean_accuracy = np.mean(accuracy_scores)
        std_accuracy = np.std(accuracy_scores)
        results[name] = (mean_accuracy, std_accuracy)
    
    return results

# Beispiel-Daten vorbereiten
# Zielvariable ist nun der direkte Teamplatz (1 bis 20)

#mean_league_team['target'] = mean_league_team['team_placement']

# Features und Zielvariable trennen
#x = mean_league_team[['age', 'height', 'weight']]
#y = mean_league_team['target']



# Liste von Modellen, die verglichen werden sollen
models = [
    ("Logistic Regression", LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')),
    ("Random Forest", RandomForestClassifier(random_state=0)),
    ("DecisionTreeClassifier",DecisionTreeClassifier(random_state=0)),
    ("SGDClassifier",SGDClassifier(random_state=0)),
    ("LGBMClassifier",LGBMClassifier(random_state=0)),
    ("XGBClassifier",XGBClassifier(random_state=0)),
    ("SVC",SVC(random_state=0))
]

# Kreuzvalidierung durchführen und Ergebnisse speichern
results = cross_validate_model(models, x, y)

# Ergebnisse anzeigen
for name, (mean_accuracy, std_accuracy) in results.items():
    print(f"Model: {name}")
    print(f"Mean Accuracy: {mean_accuracy:.4f}")
    print(f"Standard Deviation: {std_accuracy:.4f}\n")

Model: Logistic Regression
Mean Accuracy: 0.0505
Standard Deviation: 0.0335

Model: Random Forest
Mean Accuracy: 0.0579
Standard Deviation: 0.0346

Model: DecisionTreeClassifier
Mean Accuracy: 0.0536
Standard Deviation: 0.0349

Model: SGDClassifier
Mean Accuracy: 0.0513
Standard Deviation: 0.0363

Model: LGBMClassifier
Mean Accuracy: 0.0448
Standard Deviation: 0.0290

Model: XGBClassifier
Mean Accuracy: 0.0535
Standard Deviation: 0.0337

Model: SVC
Mean Accuracy: 0.0659
Standard Deviation: 0.0351



**Der Ansatz, die Modelle basierend auf dem Median von Alter, Gewicht und Größe zu berechnen, hat nicht funktioniert, unabhängig davon, ob die Daten skaliert wurden oder nicht. Um bessere Ergebnisse zu erzielen, benötigen wir neue Features.**

**Wir können feststellen durch Ergebnisse, dass die accuracy sehr schlechte Resultate zeigt. Die Ergebnisse sind so unbrauchbar, dass es sich nicht mehr lohnt, auf die anderen Metriken zu schauen.  Wir müssen in diesem Fall noch mal unser Dataset untersuchen und nach neuen Features suchen. Jede Modifikation in unserem Dataset könnte unser Modell verbessern.**